In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from iris import irisRequests
import ngl
import datetime
from numba import njit
from numba.typed import Dict
from sklearn.metrics.pairwise import haversine_distances

In [5]:
@njit
def construct_aftershocks_map(region, coords, dlat, dlon):
    N_lat = int((region[1]-region[0])/dlat)
    N_lon = int((region[3]-region[2])/dlon)
    shocks_map = np.zeros((N_lat, N_lon))
    for coord in coords:
        i,j = int((coord[0]-region[0])/dlat), int((coord[1]-region[2])/dlon)
        shocks_map[i,j] += 1
        #if(i>0):
        #    shocks_map[i-1,j] += 1
        #if(j>0):
        #    shocks_map[i,j-1] += 1
        #if(i < N_lat-1):
        #    shocks_map[i+1,j] += 1
        #if(j < N_lon-1):
        #    shocks_map[i,j+1] += 1
    return shocks_map

In [11]:
rootpath = "csv_24/"

In [12]:
regions = {}
regions['california'] = (30, 41, -125, -113)
regions['japan'] = (20, 50, 120, 150)
regions['italy'] = (35,46,6, 19)

In [13]:
regions #cosa rappresentano sti numeri ?

{'california': (30, 41, -125, -113),
 'japan': (20, 50, 120, 150),
 'italy': (35, 46, 6, 19)}

In [33]:
station_list = ngl.ngl_process_list(ngl.ngl_24h_2w) # daily measurements, with 2 weeks delay
station_list #come accedere a un element ?

,name,lat,lon,height,x,y,z,begin,end,Dtmod,NumSol,StaOrigName
0,00NA,-12.4666,130.8440,104.851,-4.073662e+06,4.712065e+06,-1.367875e+06,2008-03-27,2018-09-25,2022-05-10,3190,NaN
1,01NA,-12.4782,130.9820,105.409,-4.084823e+06,4.702027e+06,-1.369126e+06,2008-04-08,2019-09-29,2022-05-10,2362,NaN
2,02NA,-12.3559,130.8817,117.652,-4.078496e+06,4.711380e+06,-1.355915e+06,2008-09-22,2016-12-31,2022-05-10,1913,NaN
3,0ABI,68.3543,18.8164,431.388,2.233557e+06,7.610804e+05,5.906186e+06,2009-06-11,2023-09-14,2023-10-01,5189,NaN
4,0ABN,65.0337,21.3329,52.762,2.514610e+06,9.820672e+05,5.759344e+06,2023-04-05,2023-09-14,2023-10-01,162,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
21031,ZWOL,52.5024,6.0956,86.657,3.868815e+06,4.131543e+05,5.037094e+06,2005-08-02,2008-11-23,2022-05-18,1116,NaN
21032,ZYRA,52.0548,20.4203,157.400,3.683270e+06,1.371281e+06,5.006677e+06,2014-08-13,2022-12-06,2022-12-17,2144,NaN
21033,ZYWE,49.6882,19.1974,405.901,3.904715e+06,1.359564e+06,4.840731e+06,2014-04-15,2015-02-21,2022-05-18,283,NaN
21034,ZYWI,49.6867,19.2060,412.779,3.904633e+06,1.360192e+06,4.840631e+06,2003-07-24,2023-09-16,2023-09-26,6824,NaN


In [44]:
station_info = {}
for name, region in regions.items():
    station_names, station_lats, station_lons  = ngl.get_all_stations_box(station_list, *region)
    station_info[name]= (station_names, station_lats, station_lons)

#station_info['california'] has names of all stations in california, their latitudes, their longitudes

In [ ]:
station_data = {} 
for name, s_info in station_info.items():
    for s_cnt, s_name in enumerate(s_info[0]):
        df, status = ngl.ngl_retrieve_24h(rootpath, s_name)
        print(s_name, status)
        station_data[s_name] = df
#station_data[one station] contains data of earthquakes shown in some yet to be understood manner

In [50]:
station_data['5PTS']

,site,YYMMMDD,yyyy.yyyy,__MJD,week,d,reflon,e0,east,n0,...,sigma_e,sigma_n,sigma_u,corr_en,corr_eu,corr_nu,lat,lon,height,date
0,5PTS,10MAY01,2010.3299,55317,1581,6,-120.3,3140,0.612711,4033165,...,0.000688,0.000842,0.002864,0.014776,0.047809,-0.202390,36.429179,59.735023,56.51416,2010-05-01
1,5PTS,10MAY02,2010.3326,55318,1582,0,-120.3,3140,0.610349,4033165,...,0.000698,0.000853,0.002898,-0.001460,0.072677,-0.179667,36.429179,59.735023,56.51289,2010-05-02
2,5PTS,10MAY03,2010.3354,55319,1582,1,-120.3,3140,0.612365,4033165,...,0.000714,0.000858,0.002936,0.001436,0.059600,-0.158750,36.429179,59.735023,56.50849,2010-05-03
3,5PTS,10MAY04,2010.3381,55320,1582,2,-120.3,3140,0.613537,4033165,...,0.000695,0.000859,0.002850,-0.006081,0.054748,-0.149112,36.429179,59.735023,56.50528,2010-05-04
4,5PTS,10MAY05,2010.3409,55321,1582,3,-120.3,3140,0.612005,4033165,...,0.000685,0.000846,0.002808,0.016796,0.033864,-0.173429,36.429179,59.735023,56.50811,2010-05-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,5PTS,11MAY05,2011.3402,55686,1634,4,-120.3,3140,0.595514,4033165,...,0.000652,0.000824,0.002731,-0.053849,0.057743,-0.172201,36.429179,59.735023,56.55417,2011-05-05
331,5PTS,11MAY06,2011.3429,55687,1634,5,-120.3,3140,0.592347,4033165,...,0.000662,0.000830,0.002801,-0.058688,0.082758,-0.193948,36.429179,59.735023,56.55905,2011-05-06
332,5PTS,11MAY07,2011.3457,55688,1634,6,-120.3,3140,0.593752,4033165,...,0.000654,0.000824,0.002761,-0.026467,0.032168,-0.191935,36.429179,59.735023,56.56684,2011-05-07
333,5PTS,11MAY08,2011.3484,55689,1635,0,-120.3,3140,0.594539,4033165,...,0.000672,0.000848,0.002872,-0.022570,0.032551,-0.191804,36.429179,59.735023,56.56123,2011-05-08


In [51]:
start_time = datetime.datetime(2012, 1, 1, 0, 0, 0) #year, month, day, hour, minute, second
end_time =   datetime.datetime(2024, 1, 1, 0, 0, 0)
catalogs = {}
for name, region in regions.items():
    download_url = irisRequests.url_events_box(start_time, end_time, region[0], region[1], region[2], region[3], minmag=3, magtype="MW")
    df = pd.read_csv(download_url, sep="|", comment="#", parse_dates=["Time"])
    df.sort_values(by="Time", inplace=True)
    df.reset_index(inplace=True, drop=True)
    catalogs[name] = df #non capisco cosa fa questa funzione

In [53]:
catalogs['california']

,EventID,Time,Latitude,Longitude,Depth,Author,Catalog,Contributor,ContributorID,MagType,Magnitude,MagAuthor,EventLocationName
0,3333825,2012-01-02 21:11:27+00:00,31.686000,-115.932400,12.10,ISC,ISC,ISC,03896414,ML,3.00,PAS,"BAJA CALIFORNIA, MEXICO"
1,3332483,2012-01-03 14:18:55+00:00,33.209100,-119.415900,15.00,ISC,ISC,ISC,03896495,mb,3.80,ISC,SOUTHERN CALIFORNIA
2,3332587,2012-01-06 02:14:07+00:00,34.371600,-119.804400,18.20,ISC,ISC,ISC,04238987,ML,3.00,PAS,SOUTHERN CALIFORNIA
3,3332588,2012-01-06 02:47:51+00:00,34.002000,-116.337000,4.90,NEIC,ISC,ISC,03205712,ML,3.00,ANF,SOUTHERN CALIFORNIA
4,3333081,2012-01-06 06:50:10+00:00,36.002000,-117.908000,6.30,NEIC,ISC,ISC,03205728,ML,3.30,ANF,CALIFORNIA-NEVADA BORDER REGION
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5589,11751121,2023-09-30 15:26:26+00:00,40.558833,-124.288667,17.88,"at,ew,nc,us",NEIC PDE,nc,ew1696087590,Mwr,4.70,nc,NEAR COAST OF NORTHERN CALIF.
5590,11751130,2023-09-30 17:16:47+00:00,40.501000,-124.438500,29.09,"nc,ew,us",NEIC PDE,nc,ew1696094210,Mw,4.90,NC,NEAR COAST OF NORTHERN CALIF.
5591,11751372,2023-10-01 15:41:29+00:00,40.295167,-124.287000,9.80,"ew,nc,at,us",NEIC PDE,nc,ew1696174890,mw,4.09,NC,NEAR COAST OF NORTHERN CALIF.
5592,11751403,2023-10-01 19:29:36+00:00,40.291500,-124.290500,9.59,"nc,us",NEIC PDE,nc,nc73944271,ML,3.10,us,NEAR COAST OF NORTHERN CALIF.


In [ ]:
@njit(nogil=True)
def construct_map(u, idx, d, d_cutoff = 0.01):
    # u = (N_s, 3)
    # idx = (N_s)
    # d = (N_lat, N_lon, N_s_tot)
    u_map = np.zeros((d.shape[0], d.shape[1], 3))
    for i in range(0, d.shape[0]):
        for j in range(0, d.shape[1]):
            cnt = 0
            for i_n,n in enumerate(idx):
                d_ijn = d[i,j,n]
                if(d_ijn <= d_cutoff):
                    u_map[i,j,:] = u_map[i,j,:] + u[i_n, :]
                    cnt += 1
            if(cnt > 0):
                u_map[i,j,:] = u_map[i,j,:]/float(cnt)
            else:
                u_map[i,j,0] = np.nan
                u_map[i,j,1] = np.nan
                u_map[i,j,2] = np.nan
    return u_map


In [ ]:
maximal_time_shift = np.timedelta64(7, 'D')
minimal_time_shift = np.timedelta64(1, 'D')
dlat = 0.25
dlon = 0.25
for name, catalog in catalogs.items():
    print(name)
    # discretization of the region
    region = regions[name]
    N_lat = int((region[1]-region[0])/dlat)
    N_lon = int((region[3]-region[2])/dlon)
    grid_latlat, grid_lonlon = np.meshgrid( region[0] + np.arange(0, N_lat)*dlat, region[2] + np.arange(0, N_lon)*dlon, indexing='ij')
    grid_latlat = grid_latlat.flatten()
    grid_lonlon = grid_lonlon.flatten()
    grid = np.hstack([grid_latlat[:,None], grid_lonlon[:,None]])
    stations_coords = np.hstack([station_info[name][1][:,None], station_info[name][2][:,None]])
    grid_stations_dists = haversine_distances(np.radians(grid), np.radians(stations_coords))
    grid_stations_dists = grid_stations_dists.reshape((N_lat, N_lon, -1))
    

    # earthquakes identification
    large_shocks = catalog[catalog.Magnitude >= 5.8]
    large_shocks_days = large_shocks.Time
    large_shocks_coords =  large_shocks[['Latitude','Longitude']].values
    for day_time, coord in zip(large_shocks_days, large_shocks_coords):
        print(day_time)
        day = np.datetime64(datetime.datetime(day_time.year, day_time.month, day_time.day))
        aftershocks = catalog[(catalog.Time.values.astype('datetime64[D]') >=  day + minimal_time_shift)*(catalog.Time.values.astype('datetime64[D]') <= pd.to_datetime(day) +maximal_time_shift ) ]
        aftershocks_coords = aftershocks[['Latitude','Longitude']].values
        aftershocks_map = construct_aftershocks_map(region, aftershocks_coords, dlat, dlon)
        # daily data
        data = []
        indices = []
        for cnt, (s_name, _,__) in  enumerate(zip(*station_info[name])):
            df = station_data[s_name]
            row = df[df.date==  day]
            row_p = df[df.date==  day - np.timedelta64(1, 'D')]
            if(len(row) > 0 and len(row_p) > 0):
                u_n = row.lat.values[0]-row_p.lat.values[0]
                u_e = row.lon.values[0]-row_p.lon.values[0]
                u_v = row.height.values[0] - row_p.height.values[0]
                data.append((u_n, u_e, u_v))
                indices.append(cnt)
        if(len(indices) > 0):
            data = np.array(data)
            indices = np.array(indices)
            u_map = construct_map(data, indices, grid_stations_dists)
            #print(data)
            fig, ax = plt.subplots(ncols=1, figsize=(3,3))
            ax.contourf(np.sqrt(u_map[:,:,0]**2+u_map[:,:,1]**2), cmap='hsv')
            locs = np.argwhere(aftershocks_map>0)
            #ax[0].contour(aftershocks_map, cmap='Greys', levels=2)
            ax.scatter(locs[:,1], locs[:,0], color='black', s=5)
            plt.show()
            print(len(indices))